Regression OFT takes the OFG encoding and flattens it.
<br>
<br>
This script generates the OFT for prediction classification target 'ea4'.

In [28]:
import os

os.chdir("/home/tim/Development/OCPPM/")
import pandas as pd
import pm4py
import pm4py.read
import category_encoders as ce
import copy

In [29]:
ocel_in_file = "data/CS/source/cs_january_ots[krs_krv_cv]_oas[oa1_oa2_oa3_oa4_oa5_oa6_oa7_oa8_oa9_oa10_oa11_oa12_oa13_oa15_oa16]_eas[resourceCE_resourceMulti_ea1_ea2_ea3_ea4_ea6_ea8_ea10_ea12_ea14].jsonocel"
oft_out_file = "data/CS/feature_encodings/baselines/OFT/objects_w_ea4.csv"

In [30]:
# load OCEL
ocel = pm4py.read.read_ocel(ocel_in_file)
objects_df = copy.copy(ocel.objects)

In [33]:
# IDEA: add functionality to OCPA to add eas to objects as oas with an aggregation function
# select events that contain the target variable 'ea4'
events_to_take_target_from = (
    ocel.relations.sort_values("ocel:timestamp", ascending=False)
    .groupby("ocel:oid")
    .tail(1)[["ocel:eid", "ocel:oid"]]
)
# select the target variables for those events
events_with_target = ocel.events.loc[
    ocel.events["ocel:eid"].isin(events_to_take_target_from["ocel:eid"]),
    ["ocel:eid", "event_ea4"],
]
# select the objects relating to those events and their target variables
objects_with_target = events_to_take_target_from.join(
    events_with_target.set_index("ocel:eid"), on="ocel:eid"
).drop(columns=["ocel:eid"])
# join full objects table with these objects of which we now have target variable 'ea4'
objects_df_with_target = objects_with_target.join(
    objects_df.set_index("ocel:oid"), on="ocel:oid"
).set_index("ocel:oid")
objects_df_with_target = objects_df_with_target.drop(columns=["ocel:type"])

In [34]:
objects_df_with_target.to_csv(oft_out_file, index=False, sep=";")